In [1]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import json
import os
import pandas as pd # pandas 추가
from dotenv import load_dotenv

load_dotenv()

# --- 환경 변수 설정 (2단계에서 설정한 값과 동일하게) ---
# 이 부분은 실제 값을 채워 넣거나, 환경 변수로 설정해야 합니다.
service_name = os.getenv("AZURE_SEARCH_SERVICE_NAME")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") # 실제 키로 변경 필요

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") # 실제 엔드포인트로 변경 필요
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY") # 실제 키로 변경 필요
embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
llm_model_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME") # 배포된 LLM 모델 이름 (예: gpt-4, gpt-35-turbo)

# 환경 변수가 설정되지 않았다면 사용자에게 안내
if "YOUR_" in key or "YOUR_" in azure_openai_endpoint or "YOUR_" in azure_openai_key:
    print("🚨 경고: Azure 서비스 키 또는 엔드포인트가 설정되지 않았습니다.")
    print("   위 코드의 'YOUR_...' 부분을 실제 값으로 변경하거나, 환경 변수를 설정해주세요.")
    # exit() # 실제 운영 시에는 종료하거나 다른 방식으로 처리

# Azure OpenAI 클라이언트 초기화
openai_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    api_version="2024-12-01-preview"
)

# 임베딩 생성 함수
def generate_embeddings(text):
    try:
        response = openai_client.embeddings.create(
            input=text,
            model=embedding_model_name
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"❌ 임베딩 생성 중 오류 발생: {e}")
        print("   Azure OpenAI 엔드포인트, 키, 모델 배포 이름이 올바른지 확인해주세요.")
        return None
    
# Azure AI Search 클라이언트 초기화
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(key))

In [ ]:
new_project_description="디즈니 프로젝트"

new_description_vector = generate_embeddings(new_project_description)

# 2. Azure AI Search에서 유사 프로젝트 검색 (RAG)
vector_query = VectorizableTextQuery(#VectorizedQuery(
    text=new_project_description,
    k_nearest_neighbors=3, # 최상위 3개 프로젝트 탐색
    fields="text_vector"
)

results = search_client.search(
    search_text=new_project_description, # 벡터 검색만 수행하므로 텍스트 검색은 비활성화
    vector_queries=[vector_query],
    # select=["project_name", "description", "dev_parts_effort"],
    top=3 # 상위 3개 결과만 가져오기
)

In [9]:
docs = [r for r in results]

In [6]:
docs[0].keys()

dict_keys(['KOS_Con_MM', 'text_vector', 'MEIN_MM', 'ProjectName', 'APP_MM', 'parent_id', 'chunk_id', 'chunk', 'ProjectID', 'APILink_MM', 'KOS_Order_MM', 'title', 'Bill_MM', '@search.score', '@search.reranker_score', '@search.highlights', '@search.captions'])

In [10]:
docs[0].keys()

dict_keys(['KOS_Con_MM', 'text_vector', 'MEIN_MM', 'ProjectName', 'APP_MM', 'parent_id', 'chunk_id', 'chunk', 'ProjectID', 'APILink_MM', 'KOS_Order_MM', 'title', 'Bill_MM', '@search.score', '@search.reranker_score', '@search.highlights', '@search.captions'])

In [3]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") # 실제 엔드포인트로 변경 필요
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY") # 실제 키로 변경 필요

# Azure OpenAI 클라이언트 초기화
openai_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
   api_version="2024-12-01-preview",
)


In [4]:
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_completion_tokens=800,
    temperature=1.0,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    model="gpt-4.1"

)

print(response.choices[0].message.content)

That’s wonderful—Paris is a city full of iconic sights, beautiful neighborhoods, art, and amazing food! Here are some must-see highlights and ideas for what to do:

**Essential Sights:**
1. **Eiffel Tower** – Go up for a spectacular view or enjoy a picnic on the Champ de Mars.
2. **Louvre Museum** – Home to the Mona Lisa and countless masterpieces. Tip: It's vast, so plan your visit or focus on particular sections!
3. **Notre-Dame Cathedral** – Despite the 2019 fire, you can view the exterior and explore Île de la Cité.
4. **Sainte-Chapelle** – Famous for its breathtaking stained glass windows.
5. **Sacré-Cœur Basilica** – Located atop Montmartre hill for beautiful city views.

**Beautiful Walks and Neighborhoods:**
- **Le Marais** – Trendy streets, boutiques, and cool cafés.
- **Montmartre** – Bohemian charm, artists, and quaint streets. Don’t miss Place du Tertre.
- **Latin Quarter** – Historic streets, bistros, and the beautiful Luxembourg Gardens.
- **Seine River** – Walk along the